In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import utils.utils as uti
import process.file as f
import os
import tifffile

### Checking image statistics

In [44]:
def get_stats_FOCO(file):
    df = pd.DataFrame()

    imfile = sio.loadmat(file)

    data = imfile['data']
    channels = [0,1,2,3]
    RGBW = np.squeeze(data[:,:,:, channels])

    RGBW_flat = RGBW.reshape(-1, RGBW.shape[-1])

    min = np.amin(RGBW, axis = (0,1,2))
    max = np.amax(RGBW, axis = (0,1,2))
    mean = np.mean(RGBW, axis = (0,1,2))
    std = np.std(RGBW, axis = (0,1,2))
    
    print(std)

    return min, max, mean, std

In [45]:
get_stats_FOCO('data/NP_Ray/20220529-14-16-47/Composite.mat')

[ 21.10524233 114.68456053  21.50760003  71.32167479]


(array([72, 21, 30, 22], dtype=uint16),
 array([1738, 4095, 1646, 4095], dtype=uint16),
 array([174.04605165, 111.9973158 , 107.47043733, 104.03060069]),
 array([ 21.10524233, 114.68456053,  21.50760003,  71.32167479]))

### Equalizing histogram

In [46]:
def generate_histograms(image, scale):
    
    image = np.asarray(image)

    im_flat = image.reshape(-1, image.shape[-1])

    fig, axs = plt.subplots(2,2)

    for i, ax1 in enumerate(axs):
        for j, ax in enumerate(ax1):

            hist, bins = np.histogram(im_flat[:,i*2+j], 256, [0, 256*scale] )
            cdf = hist.cumsum()
            cdf_normalized = cdf * hist.max()/cdf.max()
            ax.axvline(np.max(im_flat[:,i*2+j]),linestyle='--') 
            ax.plot(cdf_normalized, color = 'b')
            ax.hist(im_flat[:,i*2+j], bins= np.arange(256)*scale, color= 'red')
            ax.set_xlabel('color channel gray count')
            ax.set_ylabel('pixel count')
            ax.set_xlim([0,256*scale])
            ax.legend(('max value', 'cdf', 'hist'), loc = 'upper right')
    
    axs[0,0].set_title('red histogram')
    axs[0,1].set_title('green histogram')
    axs[1,0].set_title('blue histogram')
    axs[1,1].set_title('white histogram')

    plt.show()


In [47]:
def equalize_hist(RGBW, threshs):
    '''
    thresh defines value above which to perform the histogram equalization
    loop through each pixel in image and transform based on histogram equalization
    '''

    size = RGBW.shape

    RGBW_new = np.zeros(size)

    flat = RGBW.reshape(-1, RGBW.shape[-1])

    for l in range(size[3]):
        channel = flat[:,l]

        thresh = threshs[l]
        
        hist_to_eq = channel[np.where(channel>=thresh)]
        N = len(hist_to_eq)
        num_bins = 4096-thresh
        hist, bins = np.histogram(hist_to_eq, num_bins, [thresh, 4096])
        cdf = hist.cumsum()
        
        for i in range(size[0]):
            for j in range(size[1]):
                for k in range(size[2]):
                        val = RGBW[i,j,k,l]

                        if val >= thresh:
                            val_index = np.where(bins==val)
                            cum_prob = cdf[val_index]/N
                            new_val = np.round(cum_prob*(num_bins-1))+thresh

                            RGBW_new[i,j,k,l] = new_val
                        
                        else:
                            RGBW_new[i,j,k,l] = val

    return RGBW_new

In [48]:
%matplotlib qt
imfile = sio.loadmat('data/NP_Ray/20220529-14-16-47/Composite.mat')
data = imfile['data']
channels = [0,1,2,3]
RGBW = np.squeeze(data[:,:,:, channels])

#generate_histograms(RGBW, 16)

RGBW = RGBW.astype('int32')
RGBW_shift = np.maximum(np.zeros(RGBW.shape), RGBW-80)
RGBW_flat = RGBW.reshape(-1, RGBW.shape[-1])
RGBW_new = equalize_hist(RGBW_shift, [145, 240, 90, 50])

In [49]:
min = np.amin(RGBW_new, axis = (0,1,2))
max = np.amax(RGBW_new, axis = (0,1,2))
mean = np.mean(RGBW_new, axis = (0,1,2))
std = np.std(RGBW_new, axis = (0,1,2))

print(std)

[233.63164781 213.84044504 251.20087559 404.55086801]


In [50]:
sio.savemat('data/NP_Ray/20220529-14-16-47/'+'hist_equal_image.mat', {'Hist_RGBW':RGBW_new})

### Convert to tiff

In [51]:
imfile = sio.loadmat('data/NP_Ray/20220529-14-16-47/'+'hist_equal_image.mat')
im = np.transpose(imfile['Hist_RGBW'],(2,3,0,1))
im = im.astype('uint16')
print(im.dtype)

tifffile.imwrite('data/NP_Ray/20220529-14-16-47/'+'hist_equal_image.ome.tif', im, imagej = True)

uint16


### Update blobs.csv with new annotations

In [ ]:
for folder in os.listdir('data/Manual_annotate'):
    for f in os.listdir('data/Manual_annotate/'+folder):
        if f == 'neuroPAL_image.csv':
            NP_output = pd.read_csv('data/Manual_annotate/'+folder+'/'+f, skiprows = 7)

        elif f == 'blobs.csv':
            blobs = pd.read_csv('data/Manual_annotate/'+folder+'/'+f)

        for i, row in NP_output.itterows():
            ID = row['User ID']
            blobs.at[i, 'ID'] = ID

    blobs.to_csv('data/Manual_annotate/'+folder+'/blobs.csv')